In [86]:
!pip install openai
!pip install -U scikit-learn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.

In [87]:
import openai
import os

import pandas as pd
import numpy as np

from pathlib import Path
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split

In [88]:
res = []

with open("./corpus__fasttext.txt","r") as f :
    for line in f:
        label, sentence = line.split(" ",1)   
        res.append((label.replace("__label__",""), sentence))    
        
df = pd.DataFrame(res,columns=["label","text"])

df

,label,text
0,radical,"""Êtes-vous prêt à faire rendre des comptes à ..."
1,radical,"""Inapproprié [malade] pour vous ? Normal pour ..."
2,radical,"""Les termes «Cabale» et «État profond» font ré..."
3,radical,"""Le pire, c'est qu'ils adorent Satan et commet..."
4,radical,"""Symboles et logos utilisés par les pédophiles..."
...,...,...
1470,non_radical,"""Les crises économique et terroriste de 2008 e..."
1471,non_radical,"""à la différence de la crise financière de 200..."
1472,non_radical,"""Dans l’instant, la photographie de la situati..."
1473,non_radical,"""Ce recul relativement faible du revenu des mé..."


In [89]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

load_dotenv(dotenv_path=Path('./.env'))

openai.api_key = os.getenv('openAI_API_KEY')
model_engine = "text-davinci-002"

predictions = []
for text in test_data['text']:
    prompt = f"Soit les 2 catégories suivantes : radical et non_radical. Classifie le texte qui suit et donne juste le nom de sa classe: {text}"
    response = openai.Completion.create(model=model_engine,temperature=0.6,prompt=prompt)
    predicted_class = "radical" if not "non" in response.choices[0].text.strip().lower() else "non_radical"
    predictions.append(predicted_class)



RateLimitError: Rate limit reached for default-text-davinci-002 in organization org-5FOvNqSe7eBGuDHnMvLBdvby on requests per min. Limit: 60 / min. Please try again in 1s. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.

In [ ]:
actual_classes = test_data['label'].to_list()

correct_predictions = 0
total_predictions = len(predictions)

for i in range(total_predictions):
    if predictions[i] == actual_classes[i]:
        correct_predictions += 1

accuracy = correct_predictions / total_predictions
print("Accuracy:", accuracy)

Accuracy: 0.4745762711864407
